## Loading necessary modules

In [43]:
import pandas as pd
import datetime
import numpy as np
from datetime import timedelta
import geocoder

## Use Ticketmaster Discovery API to get the event data

In [10]:
g = geocoder.ip('me')
print(g.latlng)

[38.4829, -121.64]


In [11]:
today = datetime.date.today()
endDate = today + timedelta(days=5)
today_str = today.strftime('%Y-%m-%dT%H:%M:%SZ')
endDate_str = endDate.strftime('%Y-%m-%dT%H:%M:%SZ')

In [12]:
mykey="Abiv3plK1RopGuqNsk8eduAiyvhTVPdI"
import ticketpy
tm_client = ticketpy.ApiClient(mykey)

pages = tm_client.events.find(
    #latlong=g.latlng,
    state_code='CA',
    start_date_time=today_str,
    end_date_time=endDate_str,
)
## test the API

In [16]:
def extract_cont(x,label):
    if(label in x.keys()):
        return(x[label])
    else:
        return(None)
def extract_information(event):
    
    event_content=event.json## transfrom event to json 
    
    ## general information of this event
    name=extract_cont(event_content,"name") 
    url=extract_cont(event_content,"url") 
    image=extract_cont(event_content["images"][0],"url")
    '''
    ## somtimes no end time
    time_start=extract_cont(time,"start")
    time_end=extract_cont(time,"end")
    if(time_start):
        time_start=time_start["localDate"]
    else:
        time_start=None
    
    if(time_end):
        time_end=time_end["localDate"]
    else:
        time_end=None
    '''   
    classifcation =extract_cont(event_content,"classifications")
    if( classifcation ):
        classifcation=classifcation[0]
    else:
        return(None)
    genre=extract_cont(classifcation,"genre")
    if(genre):
        genre=genre["name"]
    else:
        genre=None
    category=extract_cont(classifcation,"segment")
    
    if(category):
        category=category["name"]
    else:
        category=None
    ## extract location 

    venue=event_content['_embedded']["venues"][0]
    lat=venue["location"]['latitude']
    lon=venue["location"]['longitude']
    city=venue["city"]['name']

    place=venue['name']
    return({ "name":name,"link":url,"image":image,"genre":genre,"category":category,
            #"TimeStart":time_start,"TimeEnd":time_end,
            "latitude":lat,'longtitude':lon,"city":city,"place":place})

In [28]:
r=[]
for page in pages:
    for event in page:
        r.append(event)

In [32]:
r2=[]
for event in r:
    t=extract_information(event)
    if(t!=None):
        r2.append(t)
data=pd.DataFrame(r2)
data=data.loc[data["category"]!="Undefined"] ## remove undefined category

In [192]:
indices=data.groupby("category").indices

## Implement the prototype of EXP3 alg and test it(appox the true reward/preference)

In [118]:
def update_prob(gamma,weight):
    return((1-gamma)*weight/sum(weight) + gamma/len(weight))

In [127]:
def Exp3_simulator(max_iteration,gamma,options,true_rewards):
    
    def reward():
        print("Like it or Not? Y/N")
        x=input()
        if(x=="Y"):
            return(1)
        elif(x=="N"):
            return(-1)
        else:
            return(0)
    
    num = len(options)
    
    weights = np.ones(num) ## initial weights are the same for all options
    weights=weights/sum(weights)
    
    for t in range(max_iteration): ### maximum run for 1000 loops
        
        prob = update_prob(gamma, weights)
        
        choice=np.random.choice(num, 1, p=prob) ## randomly choose one option based on the caculated probability
        
        if(len(true_rewards)>0): ## exist the true_reward 
            
            theReward = 2*np.random.binomial(n=1, p=true_rewards[choice]) -1 ## 0/1 response to -1/+1
            ## randomly generate reward based on true reward
        else:
            print(choice)
            theReward = reward()
            ## get reward from input 

        estimatedReward = 1.0 * theReward / prob[choice]
        weights[choice] = weights[choice]*np.exp(estimatedReward * gamma / num)
        # important that we use estimated reward here!
    return(weights)

In [124]:
x=range(5)
N=len(x)
truth = np.random.uniform(low=0, high=1, size=N) # 
y=Exp3_simulator(1000,0.07,x,truth)

In [125]:
print("True_reward:")
print(truth)
print("Estimated_reward:")
print(y)

True_reward:
[0.42946645 0.91542257 0.86469688 0.4473021  0.28995334]
Estimated_reward:
[9.68934976e-02 4.17519337e+04 2.03359540e+03 1.15326025e-02
 6.50409561e-04]


Instead of using true reward and randomly generated reward.
It can also get the reward from screen standard input (Like 1, Don't Like  -1).  
Here I only input Y(like) when recommend 3,4, as we can see the estimated reward for them increased. 

In [159]:
x=range(5)
N=len(x)
truth = np.random.uniform(low=0, high=1, size=N) # 
y=Exp3_simulator(10,0.1,x,true_rewards=[])

[0]
Like it or Not? Y/N
N
[1]
Like it or Not? Y/N
N
[0]
Like it or Not? Y/N
N
[2]
Like it or Not? Y/N
N
[4]
Like it or Not? Y/N
Y
[4]
Like it or Not? Y/N
Y
[1]
Like it or Not? Y/N
N
[3]
Like it or Not? Y/N
Y
[1]
Like it or Not? Y/N
N
[4]
Like it or Not? Y/N
Y


In [160]:
print("Estimated_reward:")
print(y)

Estimated_reward:
[0.16281966 0.14514375 0.1818854  0.21998422 0.25979202]


### Implement it on our dataset got from TickmasterAPI 

In [214]:
def recommend(event_indices,data):
    n=len(event_indices)
    choice=event_indices[np.random.choice(n,1)[0]]
    event=data.iloc[[choice],:]
    print(event[["name","city","place","link","category","genre"]])

In [215]:
def reward():
        print("Like it or Not? Y/N")
        x=input()
        if(x=="Y"):
            return(1)
        elif(x=="N"):
            return(-1)
        else:
            return(0) ## here if input any other breaks the recommender system

In [216]:
gamma=0.1 ## set the speed parameter = 0.1
options=list(indices.items())## transform it into list for indexing

In [217]:
Num=len(indices)
weights = np.ones(Num)
while(True):
    
    weights=weights/sum(weights)
    probability =update_prob(gamma, weights)
    choice=np.random.choice(Num, 1, p=probability)[0]
    ## generate event by choice
    print("Recommend: "+ options[choice][0]) ## catrgory of recommendation
    recommend(options[choice][1],data)
    theReward = reward()  ## each time only update the weight
    if(theReward==0):
        break
    estimatedReward = 1.0 * theReward / probability[choice]
    weights[choice] = weights[choice]*np.exp(estimatedReward * gamma / Num)

Recommend: Arts & Theatre
            name  city        place  \
71  Chris Porter  Brea  Brea Improv   

                                                 link        category   genre  
71  https://www.ticketweb.com/event/chris-porter-b...  Arts & Theatre  Comedy  
Like it or Not? Y/N
Y
Recommend: Film
                                     name           city           place  \
292  2001: a Space Odyssey New 70mm Print  San Francisco  Castro Theater   

                                                  link category          genre  
292  https://www.ticketweb.com/event/2001-a-space-o...     Film  Miscellaneous  
Like it or Not? Y/N
H


In [221]:
s=data.iloc[2,:]

In [222]:
s["link"]

'http://www.ticketsnow.com/InventoryBrowse/TicketList.aspx?PID=2318144'

### Implement it on our dataset got from TickmasterAPI  using Tkinster to get the resopnse(GUI Program)

The code for the GUI implementation is here for review convienece, the actual working code are in .py file in the 

In [ ]:
### GUI window design using tkinter 

In [211]:
np.random.choice(5, 1, p=probability)

array([0])

In [212]:
probability

array([0.2102663 , 0.21248291, 0.15671818, 0.2102663 , 0.2102663 ])

In [213]:
?np.random.choice

In [284]:
df=pd.read_csv("Video_Games_Sales_as_at_22_Dec_2016.csv")
data=df.loc[:,["Name","Platform","Year_of_Release","Genre","Rating"]]
ind=data.groupby(["Platform"])["Name"].count()

In [334]:
data2=data.set_index("Platform").loc[ind_pl,:].reset_index()
ind_pl=list(ind.index[ind>500])
indices=data2.groupby("Genre").indices
options=list(indices.items())## transform it into list for indexing

In [335]:
data2["Year_of_Release"]=data2["Year_of_Release"].fillna("NaN.0")

In [338]:
t=data2["Year_of_Release"][14301]

In [339]:
t[:-2]

'NaN'

In [337]:
np.delete(s,0)

array([   77,   120,   136, ..., 14293, 14313, 14320])

In [308]:
options[0]=list(options[0])

In [310]:
[ list(x) for x in options]

[['Action', array([    9,    10,    14, ..., 14316, 14322, 14325])],
 ['Adventure', array([   53,    77,   120, ..., 14293, 14313, 14320])],
 ['Fighting',
  array([    6,    41,   105,   131,   171,   190,   265,   281,   295,
           321,   336,   371,   464,   466,   706,   774,   796,   820,
           844,   884,   930,   935,   943,  1000,  1144,  1231,  1269,
          1303,  1322,  1346,  1388,  1505,  1639,  1652,  1701,  1705,
          1759,  1774,  1825,  1828,  1830,  1995,  2013,  2023,  2078,
          2082,  2116,  2133,  2242,  2312,  2738,  2824,  2848,  2943,
          2955,  2979,  3024,  3097,  3105,  3133,  3135,  3187,  3213,
          3222,  3238,  3251,  3279,  3321,  3365,  3403,  3465,  3470,
          3490,  3494,  3519,  3538,  3557,  3563,  3566,  3590,  3598,
          3600,  3612,  3624,  3637,  3639,  3644,  3655,  3665,  3681,
          3695,  3708,  3714,  3716,  3729,  3742,  3788,  3800,  3803,
          3814,  3824,  3826,  3850,  3857,  3862,  3